In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import tensorflow.keras.layers as L
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import torch
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
# from bs4 import BeautifulSoup
import re,string,unicodedata
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding="latin1")
test=pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_test.csv",encoding="latin1")

In [ ]:
print(train['Sentiment'][41156])

In [ ]:
stop_word = stopwords.words('english')#加载英文停止词

def clean(text):

    # 清除链接
    text = re.sub(r'http\S+', " ", text)

    # 清除@他人
    text = re.sub(r'@\w+',' ',text)

    # 清除推特话题标签
    text = re.sub(r'#\w+', ' ', text)

    # 清除数字
    text = re.sub(r'\d+', ' ', text)

    # 清除链接标签
    text = re.sub('r<.*?>',' ', text)
    
    # 清除停止词
    text = text.split()
    text = " ".join([word for word in text if not word in stop_word])
    
      
    return text

In [ ]:
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: clean(x))
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda x: clean(x))
#使用正则式初步处理数据

In [ ]:
sb.heatmap(train.isnull())

In [ ]:
sb.heatmap(test.isnull())

In [ ]:
df_train = train.iloc[:,4:]
df_test = test.iloc[:,4:]
#只需要OriginalTweet和Sentiment两列

In [ ]:
df_train.head()

In [ ]:
emo={"Extremely Negative":0,"Negative":1,"Neutral":2, "Positive":3,"Extremely Positive":4}

df_train['Sentiment'] = df_train['Sentiment'].map(emo)
df_test['Sentiment']  = df_test['Sentiment'].map(emo)


In [ ]:
print(df_train['Sentiment'])
df_train.head()

In [ ]:
corpus = df_train['OriginalTweet'].tolist()
corpus1 = df_test['OriginalTweet'].tolist()

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer()
re = tfidf2.fit_transform(corpus)
re1 = tfidf2.transform(corpus1)


In [ ]:
x_train_tfidf = re.toarray()
x_test_tfidf = re1.toarray()

In [ ]:
print(x_train_tfidf.shape)
print(x_test_tfidf.shape)

# 词袋

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

ce = vectorizer.fit_transform(corpus)
ce1 = vectorizer.transform(corpus1)

In [ ]:
x_train_bow = ce.toarray()
x_test_bow = ce1.toarray()

In [ ]:
y_train = df_train.Sentiment.tolist()
y_test = df_test.Sentiment.tolist()

In [ ]:
print(y_train[:5])

In [ ]:
print(x_train_bow.shape)
print(x_test_bow.shape)

**bayes&tfidf**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
naiveByes_clf = MultinomialNB()

naiveByes_clf.fit(x_train_tfidf,y_train)

NB_prediction = naiveByes_clf.predict(x_test_tfidf)
NB_accuracy = accuracy_score(y_test,NB_prediction)
print("training accuracy Score    : ",naiveByes_clf.score(x_train_tfidf,y_train))
print("Validation accuracy Score : ",NB_accuracy )
print(classification_report(NB_prediction,y_test))

**bayes&cbow**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
naiveByes_clf = MultinomialNB()

naiveByes_clf.fit(x_train_bow,y_train)

NB_prediction = naiveByes_clf.predict(x_test_bow)
NB_accuracy = accuracy_score(y_test,NB_prediction)
print("training accuracy Score    : ",naiveByes_clf.score(x_train_bow,y_train))
print("Validation accuracy Score : ",NB_accuracy )
print(classification_report(NB_prediction,y_test))

**SGD&CBOW**

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss = 'hinge', penalty = 'l2', random_state=0)

sgd_clf.fit(x_train_bow,y_train)

sgd_prediction = sgd_clf.predict(x_test_bow)
sgd_accuracy = accuracy_score(y_test,sgd_prediction)
print("Training accuracy Score    : ",sgd_clf.score(x_train_bow,y_train))
print("Validation accuracy Score : ",sgd_accuracy )
print(classification_report(sgd_prediction,y_test))

**SGD&tfidf**

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss = 'hinge', penalty = 'l2', random_state=0)

sgd_clf.fit(x_train_tfidf,y_train)

sgd_prediction = sgd_clf.predict(x_test_tfidf)
sgd_accuracy = accuracy_score(y_test,sgd_prediction)
print("Training accuracy Score    : ",sgd_clf.score(x_train_tfidf,y_train))
print("Validation accuracy Score : ",sgd_accuracy )
print(classification_report(sgd_prediction,y_test))

In [ ]:
embedding_vectors = 30
VOCAB_SIZE = len(corpus)+1


In [ ]:
model = tf.keras.Sequential([
    L.Embedding(VOCAB_SIZE,embedding_vectors, input_length=x_train_tfidf.shape[1]),
    L.Bidirectional(L.LSTM(256,return_sequences=True)),
    L.GlobalMaxPool1D(),
    L.Dropout(0.4),
    L.Dense(256, activation="relu"),
    L.Dropout(0.4),
    L.Dense(3)
])

In [ ]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',metrics=['accuracy']
             )

In [ ]:
model.summary()

In [ ]:
model.fit(np.array(x_train_tfidf),np.array(y_train),epochs=2,
          validation_data=(np.array(x_test_tfidf),np.array(y_test)))

SGD&tfidf

In [ ]:
Y_valid=y_test
#Y_valid = enumerate(Y_valid)
Y_pred = NB_prediction
#Y_pred = enumerate(Y_pred)


五分类

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_curve
precision = precision_score(Y_valid, Y_pred, average='micro')
recall = recall_score(Y_valid, Y_pred, average='micro')
f1_score = f1_score(Y_valid, Y_pred, average='micro')
accuracy_score = accuracy_score(Y_valid, Y_pred)
nb_classes = 5
print("Precision_score:",precision)
print("Recall_score:",recall)
print("F1_score:",f1_score)
print("Accuracy_score:",accuracy_score)
# roc_curve:真正率（True Positive Rate , TPR）或灵敏度（sensitivity）
# 横坐标：假正率（False Positive Rate , FPR）
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
print(Y_valid[0])
for i in range(nb_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_valid[:,i], Y_pred.count[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Y_valid.ravel(), Y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# Compute macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(nb_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(nb_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= nb_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
# Plot all ROC curves
lw = 2
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
label='micro-average ROC curve (area = {0:0.2f})'
''.format(roc_auc["micro"]),
color='deeppink', linestyle=':', linewidth=4)
plt.plot(fpr["macro"], tpr["macro"],
label='macro-average ROC curve (area = {0:0.2f})'
''.format(roc_auc["macro"]),
color='navy', linestyle=':', linewidth=4)
colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(nb_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
    label='ROC curve of class {0} (area = {1:0.2f})'
    ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.savefig("../images/ROC/ROC_5分类.png")
plt.show()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print(x_train_tfidf.shape)
print(x_test_tfidf.shape)
